In [6]:
from Conference import Conference
from thConference import thConference
from wikicfputil import parseConference, parsethConf
import requests
from lxml import etree
import pymongo
import re
from datetime import datetime
import time
from operator import itemgetter, attrgetter

In [11]:
# 入库
def save(data, series=False, series_ids=None):
    if series:
        assert len(data) == len(series_ids)
    client = pymongo.MongoClient('127.0.0.1:27017')
    db = client.crawled_data
    
    if series:
        # 更新series库(wikicfp表)中的events
        for e,sid in zip(data,series_ids):
            record = db.wikicfp2.find_one({'series':sid})
            if record and e.year not in [th['year'] for th in record['event']]:
                record['event'].append(e.__dict__)
                infos = sorted(record['event'], key=lambda c: c['year'], reverse=True)
                db.wikicfp2.update_one({'series':sid},{'$set':{'event':infos}})
            else: print('thConference has saved or series id not found, sid:', sid)
        pass
    else:
        # 存放到 wikicfp_update 表中
        data = [d.__dict__ for d in data]
        # 去重
        for conf in data:
            if db.wicicfp_update.find_one({'thname':conf['thname']}):
                data.remove(conf)
        db.wikicfp_update.insert_many(data)
    
    print('Save update data:',len(data))
    client.close()

def conftodict(conf):
    return {
            '_type': 'Conference',
            'name': conf.name,
            'abbr': conf.abbr,
            'series': conf.series,
            'link': conf.link,
            'main_link': conf.main_link,
            'past': conf.past,
            'future': conf.future,
            'event': [e.__dict__ for e in conf.event]
        }

In [8]:
# test_events = series_events[3:5] # 1121 2986
# test_sids = series_ids[3:5]
# # for e in test_events:
# #     e.thname="Test Test"
# save(test_events, series=True, series_ids=test_sids)

# conf = db.wikicfp_update.find_one({'series':'1121'})
# # conf['event'][0]

# 抓取更新会议
1. 先抓取全部更新  
2. 有series的放到 series数组  没有的放到events  
3. 入库(未实现)

In [9]:
urls = [r'http://www.wikicfp.com/cfp/allcfp?page='+str(i) for i in range(1,11)]
domin = r'http://www.wikicfp.com'

# allcfp网页所有最新的届会议 并封装成conf对象
series_events = []
series_ids = []
events = []

for url in urls:
    response = requests.get(url)
    page = etree.HTML(response.text)

    print('抓取更新：', url[-1],'页')

    bgcolors = ['#f6f6f6', '#e6e6e6']
    for bgcolor in bgcolors:
        a_s = page.xpath('//tr[@bgcolor="' + bgcolor + '"]/td/a')
        for a in a_s:
            year = a.text.strip().split()[-1]
            year = re.findall('\d{1,4}', year)[0]
            event,series = parsethConf(thConference(link=domin + a.attrib['href'], thabbr=a.text, year=int(year)))
            if series: 
                series_events.append(event)
                series_ids.append(series)
            else: events.append(event)

抓取更新： 1 页
抓取更新： 2 页
抓取更新： 3 页
抓取更新： 4 页
抓取更新： 5 页
抓取更新： 6 页
抓取更新： 7 页
抓取更新： 8 页
抓取更新： 9 页
抓取更新： 0 页


# 更新抓取

In [ ]:
save(series_events, series=True, series_ids=series_ids)
save(events)
events.extend(series_events)
events_abbrs = [e.thabbr for e in events]
dict_new_series_ids = {}
dict_new_series_events = {}
dict_new_events = {}
while True:
    dict_new_events.clear()
    response = requests.get(urls[0])
    page = etree.HTML(response.text)

    print('Update checking：', datetime.now())

    bgcolors = ['#f6f6f6', '#e6e6e6']
    for bgcolor in bgcolors:
        a_s = page.xpath('//tr[@bgcolor="' + bgcolor + '"]/td/a')
        for a in a_s:
            event,series = parsethConf(thConference(link=domin + a.attrib['href'], thabbr=a.text))
            if series:
                dict_new_events[event.thabbr] = event
                dict_new_series_ids[event.thabbr] = series
            else:
                dict_new_series_events[event.thabbr] = event
    
    # 非Series数据 
    new_events_abbrs = dict_new_events.keys()
    set_new_evnets_abbrs = set(new_events_abbrs) - set(events_abbrs)
    if(set_new_evnets_abbrs): 
        [print('New non-series record:', abbr) for abbr in set_new_evnets_abbrs]
        new_events = [dict_new_events[key] for key in set_new_evnets_abbrs]
        save(new_events)
    else: print('No non-series update data.')
    
    # Series数据 
    new_series_events_abbrs = dict_new_series_events.keys()
    set_new_series_evnets_abbrs = set(new_series_events_abbrs) - set(events_abbrs)
    if(set_new_series_evnets_abbrs): 
        [print('New series record:', abbr) for abbr in set_new_series_evnets_abbrs]
        new_events = [dict_new_events[key] for key in set_new_series_evnets_abbrs]
        save(new_events, series=True, series_ids=[dict_new_series_ids[k.thabbr] for k in new_events])
    else: print('No series update data.')
    
    # 更新缓存
    if set_new_evnets_abbrs or set_new_series_evnets_abbrs:
        events_abbrs = getAllAbbts()

    time.sleep(30*60)

thConference has saved or series id not found, sid: 1284
thConference has saved or series id not found, sid: 1121
thConference has saved or series id not found, sid: 2986
thConference has saved or series id not found, sid: 2706
thConference has saved or series id not found, sid: 1506
thConference has saved or series id not found, sid: 2933
thConference has saved or series id not found, sid: 1683
thConference has saved or series id not found, sid: 2904
thConference has saved or series id not found, sid: 1862
thConference has saved or series id not found, sid: 1363
thConference has saved or series id not found, sid: 665
thConference has saved or series id not found, sid: 1671
thConference has saved or series id not found, sid: 2284
thConference has saved or series id not found, sid: 447
thConference has saved or series id not found, sid: 933
thConference has saved or series id not found, sid: 2722
thConference has saved or series id not found, sid: 10
thConference has saved or series id 